In [1]:
# [셀 2] 라이브러리/경로
import os
import pandas as pd
from sqlalchemy import create_engine, text
from getpass import getpass

CSV_PATH = "./districts.csv"  # 업로드된 파일 경로


In [2]:
# [셀 3] CSV 로드 & 기본 점검
df = pd.read_csv(CSV_PATH)

print("shape:", df.shape)
print(df.dtypes)
display(df.head())

# 필수 컬럼 확인
required = {"district_id", "district_name", "district_code", "district_short_name"}
missing = required - set(df.columns)
assert not missing, f"Missing columns: {missing}"

# 결측/중복 점검
print("null counts:\n", df.isna().sum())
print("duplicate bjd_cd:", df["district_id"].duplicated().sum())


shape: (20555, 4)
district_id             int64
district_name          object
district_code           int64
district_short_name    object
dtype: object


,district_id,district_name,district_code,district_short_name
0,1100000000,서울특별시,11000,서울특별시
1,1111000000,서울특별시 종로구,11110,서울특별시 종로구
2,1111010100,서울특별시 종로구 청운동,11110,서울특별시 종로구
3,1111010200,서울특별시 종로구 신교동,11110,서울특별시 종로구
4,1111010300,서울특별시 종로구 궁정동,11110,서울특별시 종로구


null counts:
 district_id            0
district_name          0
district_code          0
district_short_name    0
dtype: int64
duplicate bjd_cd: 0


In [3]:
# [셀 4] MySQL 접속정보 입력 (안전하게 getpass 사용)
MYSQL_HOST = os.getenv("MYSQL_HOST") or input("MYSQL_HOST (예: 127.0.0.1): ").strip()
MYSQL_PORT = int(os.getenv("MYSQL_PORT") or input("MYSQL_PORT (기본 3306): ") or "3306")
MYSQL_USER = os.getenv("MYSQL_USER") or input("MYSQL_USER: ").strip()
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD") or getpass("MYSQL_PASSWORD: ")
MYSQL_DB = os.getenv("MYSQL_DB") or input("MYSQL_DB (예: mydb): ").strip()

# charset=utf8mb4 권장 (한글)
engine = create_engine(
    f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
    f"?charset=utf8mb4",
    pool_pre_ping=True,
)


In [4]:
# [셀 5] 테이블 생성 (PK 포함) - 권장 스키마
TABLE = "districts"  # 원하는 테이블명으로 바꿔도 됨

create_sql = f"""
CREATE TABLE IF NOT EXISTS `{TABLE}` (
  `district_id` BIGINT NOT NULL,
  `district_name` VARCHAR(100) NULL,
  `district_code` BIGINT NULL,
  `district_short_name` VARCHAR(100) NOT NULL,
  PRIMARY KEY (`district_id`),
  INDEX `idx_district_code` (`district_code`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
"""

with engine.begin() as conn:
    conn.execute(text(create_sql))

print("table ready:", TABLE)


table ready: districts


In [5]:
# [셀 6] 적재 방식 선택
# 옵션 A) 완전 새로 넣기(테이블 비우고 insert)
# 옵션 B) upsert(동일 bjd_cd 있으면 update)

LOAD_MODE = "A"  # "A" 또는 "B"


In [7]:
# [셀 7-A] 완전 새로 넣기(테이블 TRUNCATE 후 벌크 insert)
if LOAD_MODE.upper() == "A":


    # pandas.to_sql은 PK/인덱스는 건드리지 않고, 데이터만 insert 함
    df.to_sql(TABLE, con=engine, if_exists="append", index=False, chunksize=5000, method="multi")
    print("loaded with TRUNCATE+append:", len(df))


loaded with TRUNCATE+append: 20555


In [8]:
# [셀 8] 적재 검증 (행 수/샘플)
with engine.connect() as conn:
    n = conn.execute(text(f"SELECT COUNT(*) FROM `{TABLE}`;")).scalar_one()
    sample = conn.execute(text(f"SELECT * FROM `{TABLE}` ORDER BY district_id LIMIT 5;")).fetchall()

print("row_count_in_db:", n)
print("sample_rows:")
for r in sample:
    print(r)


row_count_in_db: 20555
sample_rows:
(1100000000, '서울특별시', 11000, '서울특별시')
(1111000000, '서울특별시 종로구', 11110, '서울특별시 종로구')
(1111010100, '서울특별시 종로구 청운동', 11110, '서울특별시 종로구')
(1111010200, '서울특별시 종로구 신교동', 11110, '서울특별시 종로구')
(1111010300, '서울특별시 종로구 궁정동', 11110, '서울특별시 종로구')
